# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine 
import re

# sklearn 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.externals import joblib
# nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/eddadsi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eddadsi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eddadsi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [82]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('Messages', con = engine)
X = df.message
Y = df.drop(labels=['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [83]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    #lemmatize and remove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens 
              if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [85]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10,
                                                         random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X.values, Y.values)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...           oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [87]:
y_pred = pipeline.predict(X_test)
class_names = list(Y.columns)
def calcul_metrics(y_test, y_pred):
    dic_metrics = {'precision' : [], 'recall' : [], 'f1-score' : []}
    for column in range(y_pred.shape[1]):
        metrics = classification_report(y_test[:, 0], y_pred[:, 0], output_dict=True)['micro avg']
        dic_metrics['precision'].append(metrics['precision'])
        dic_metrics['recall'].append(metrics['recall'])
        dic_metrics['f1-score'].append(metrics['f1-score'])
    
    # Stock metrics in a Dataframe
    df_metrics = pd.DataFrame(dic_metrics, index=class_names)
    return df_metrics.mean()

In [88]:
calcul_metrics(y_test, y_pred)

precision    0.800427
recall       0.800427
f1-score     0.800427
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [97]:
# parametres
parameters =  {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 1),
    'vect__max_features': (None, 5000),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100]
}

# GridSearchCV
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=-1) 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [98]:
cv.fit(X_train[:100], y_train[:100])
y_pred = cv.predict(X_test[:100])
calcul_metrics(y_test[:100], y_pred)

/home/eddadsi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/eddadsi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


precision    0.55
recall       0.55
f1-score     0.55
dtype: float64

In [ ]:
# print results metrics
calcul_metrics(y_test[:100], y_pred)

In [ ]:
# print model best Parameters
print("\nBest Parameters:", cv.best_params_)

### 9. Export your model as a pickle file

In [ ]:
# save the classifier
filename = 'digits_classifier.joblib.pkl'
_ = joblib.dump(cv, filename, compress=9)

In [ ]:
# load it again
clf2 = joblib.load(filename)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.